In [ ]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [1]:
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [61]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='VALID')#padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='VALID')#padding='SAME')


# Create model
def conv_net(x, weights, biases):#, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    #conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    #conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    #fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['wout']), biases['bout'])
    return out

In [62]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([10, 10, 1, 16])),#5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([10, 10, 16, 32])),#5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([10*10*32, 256])),#7*7*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'wout': tf.Variable(tf.random_normal([256, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([16])),
    'bc2': tf.Variable(tf.random_normal([32])),
    'bd1': tf.Variable(tf.random_normal([256])),
    'bout': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases)#, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [ ]:
"""import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from pylab import *

#matplotlib.use('Agg')
with tf.Session() as sess:
    sess.run(init)
    w = sess.run(weights)
    #print(w['wc1'])
    obj = plt.pcolor(np.reshape(w['wc1'][:, :, 0, 0], (25, 25)))
    set_cmap('gray')
    colorbar()
    
    plt.show()"""

In [63]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    acc = 0
    #while step * batch_size < training_iters:
    while acc < 0.8:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y})#,
                                                              #keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print ("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))
    # Backup params
    trained_weights = sess.run(weights)
    trained_biases = sess.run(biases)

Iter 1280, Minibatch Loss= 28899.382812, Training Accuracy= 0.25781
Iter 2560, Minibatch Loss= 25020.455078, Training Accuracy= 0.35156
Iter 3840, Minibatch Loss= 15073.139648, Training Accuracy= 0.57031
Iter 5120, Minibatch Loss= 13870.677734, Training Accuracy= 0.55469
Iter 6400, Minibatch Loss= 8358.896484, Training Accuracy= 0.67188
Iter 7680, Minibatch Loss= 6972.493164, Training Accuracy= 0.68750
Iter 8960, Minibatch Loss= 7517.324707, Training Accuracy= 0.71094
Iter 10240, Minibatch Loss= 10708.041016, Training Accuracy= 0.61719
Iter 11520, Minibatch Loss= 7424.993652, Training Accuracy= 0.70312
Iter 12800, Minibatch Loss= 9519.800781, Training Accuracy= 0.67969
Iter 14080, Minibatch Loss= 4374.840332, Training Accuracy= 0.75781
Iter 15360, Minibatch Loss= 5486.693359, Training Accuracy= 0.72656
Iter 16640, Minibatch Loss= 4911.049805, Training Accuracy= 0.78906
Iter 17920, Minibatch Loss= 5596.292969, Training Accuracy= 0.74219
Iter 19200, Minibatch Loss= 4321.762207, Training 

In [64]:
import json
import os
import numpy as np

def SimpleEncode(ndarray):
    return json.dumps(ndarray.tolist())

dir_name = "experiment"
# Create directory
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)
    
trained_params = {} #**trained_weights, **trained_biases}
for weight_key in trained_weights:
    if len(trained_weights[weight_key].shape) == 2:
        # MLP weights
        trained_params[weight_key] = trained_weights[weight_key].transpose()
    else:
        # Convolutional weights
        trained_params[weight_key] = trained_weights[weight_key]
trained_params = {**trained_params, **trained_biases}
    
for param_name in trained_params:
    json_string = SimpleEncode(trained_params[param_name].astype(np.float16))
    print(trained_params[param_name].shape)
    with open(dir_name + '/' + param_name + '.json', 'w') as outfile:
        outfile.write(json_string)
        outfile.close
    print("Saved param \"" + param_name + "\"")

(32,)
Saved param "bc2"
(10, 10, 16, 32)
Saved param "wc2"
(10, 10, 1, 16)
Saved param "wc1"
(256, 3200)
Saved param "wd1"
(10,)
Saved param "bout"
(16,)
Saved param "bc1"
(10, 256)
Saved param "wout"
(256,)
Saved param "bd1"


In [65]:
start = 0
end = 1

test_images, test_targets = mnist.test.next_batch(100)

test_batch_images = test_images[start:end]
test_batch_targets = test_targets[start:end]

# Save test images
json_string = SimpleEncode(test_batch_images.astype(np.float32))
with open(dir_name + '/test_images_n' + str(end-start) + '.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
    
print("Saved " + str(test_batch_images.shape[0]) + " images, shape: ", end='')
print(test_batch_images.shape)

Saved 1 images, shape: (1, 784)


In [66]:
import numpy as np


### Save Tensorflow's forward propagation results into a JSON file
print("Forward-propagating...")
n_input = 784
x = tf.placeholder("float", [None, n_input])

# Convert arrays to tensors
trained_weights_tensors = {}
for weight_name in trained_weights:
    trained_weights_tensors[weight_name] = tf.convert_to_tensor(trained_weights[weight_name].astype(np.float16).astype(np.float32))

trained_biases_tensors = {}
for bias_name in trained_biases:
    trained_biases_tensors[bias_name] = tf.convert_to_tensor(trained_biases[bias_name].astype(np.float16).astype(np.float32))
    
# Construct model
pred = conv_net(x, trained_weights_tensors, trained_biases_tensors)

# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    #run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
    #run_metadata = tf.RunMetadata()
    sess.run(init)
    # Produce outputs
    result = sess.run([pred], feed_dict={x: test_batch_images})#, options=run_options, run_metadata=run_metadata)
    # Create the Timeline object, and write it to a json
    #tl = timeline.Timeline(run_metadata.step_stats)
    #ctf = tl.generate_chrome_trace_format()
    #current_time = datetime.datetime.now()
    #if not os.path.isdir(dir_name + "/results_tensorflow"):
    #    os.mkdir(dir_name + "/results_tensorflow")
    #with open(dir_name + "/results_tensorflow/" + current_time.strftime("%d.%m.%Y-%H.%M.%S.") + 
    #          str(int(current_time.microsecond / 1000)).zfill(3) + ".n" + str(end-start) +
    #          ".timeline.json", 'w') as f:
    #    f.write(ctf)

# Print results
verbose = True
if verbose:
    np.set_printoptions(threshold=np.inf, suppress=True)
    #print("Weights[0][0]:")
    #print(trained_weights)
    i = 0
    print("Inputs[" + str(i) + "]:")
    print(test_batch_images[i].sum())
    #print("Weights['wc1'][0][0]:")
    #print(trained_weights['wc1'][0][0])
    #print("Biases['bc1']:")
    #print(trained_biases['bc1'])
    print("Weights['wd1'][0]:")
    print(trained_weights['wd1'][0])
    print("Biases['bd1']:")
    print(trained_biases['bd1'])
    print("Output[" + str(i) + "]:")
    print(result[0][i].shape)
    print(result[0][i])
    print("Output[0:" + str(i+1) + "] maxed:")
    print([list(decision).index(max(decision)) for decision in result[0][:i+1]])
    print("Correct[0:" + str(i+1) + "]:")
    print([list(decision).index(max(decision)) for decision in test_batch_targets[:i+1]])

# Save results
json_string = SimpleEncode(result[0].astype(np.float32))
with open(dir_name + '/test_tf_results_n' + str(end-start) + '.json', 'w') as outfile:
    outfile.write(json_string)
    outfile.close
if verbose:
    print("Saved results, shape: ", end='')
    print(result[0].shape)

Forward-propagating...
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Inputs[0]:
213.753
Weights['wd1'][0]:
[-0.47187206  1.08341396 -1.47720969 -0.19636898 -0.51985645  0.24864505
 -1.24037206 -0.4832938   0.00023006  0.40612677  0.08893027  0.39518809
 -0.29394531 -1.82325184 -0.23145139 -1.00753617  1.32293379  0.33850086
  1.24928176  0.69410473 -0.79484934 -0.25488058 -0.29076239  0.93654656
 -1.05741274  0.62239391  0.54844779  1.68540788  1.09214199  0.22519499
  1.0957396  -0.71046937  1.32815003 -0.15523241 -1.69996715 -0.21848178
 -0.74748725 -2.60405779 -1.58781004  0.30472574  1.17814505  0.15850191
 -0.52324247 -1.10162449  1.18046141  0.15323745  2.059165   -0.75512916
 -0.70019263  1.65853167  0.03873347 -0.24496233  0.07724958  0.40521377
  0.54079425 -2.66528678 -0.11002549 -0.2445145  -0.87325329 -0.18451075
  0.77185243  2.15242648 -0.23326361  1.63746047  0.77986103  0.9038437
 -0.82536131 -0.65774959 -0.33969179  0.42808837 -0.31729046 -0